In [ ]:
from dotenv import load_dotenv
load_dotenv

from typing_extensions import TypedDict, Annotated
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.prebuilt import tool_node, tools_condition
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
class State:
    messages : Annotated[list, add_messages]